In [322]:
import os
import random
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor

In [327]:
def process_subject(sub_id):
    df = pd.read_csv(f'annotations-rpoints/mesa-sleep-{sub_id}-rpoint.csv')
    if not np.all(np.isin(df['stage'].values, [0, 1, 2, 3, 4, 5])):
        print(f'Invalid labels for subject {sub_id}')
        return
    # convert stage 4 to stage 3
    df['stage'] = df['stage'].map({0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 4})
    df = df[['seconds', 'stage']] 
    df['IBI'] = df['seconds'].diff()
    df['IBI'] = df['IBI'].fillna(method='bfill')
    # Remove IBI values > 5 standard deviations from the mean
    df = df[np.abs(df['IBI'] - df['IBI'].mean()) <= (5 * df['IBI'].std())]
    df['IHR'] = 60 / df['IBI']
    # Normalize IHR by subtracting mean and dividing by standard deviation
    df['IHR'] = (df['IHR'] - df['IHR'].mean()) / df['IHR'].std()
    df['seconds'] = pd.to_datetime(df['seconds'], unit='s')
    # resample to 2Hz preserve the labels as integers
    df = df.resample('500L', on='seconds').mean()
    # interpolate missing values using linear interpolation
    df = df.interpolate(method='linear')
    # create a new column for labels
    df = df[['IHR', 'stage']]
    # check is there is a df folder in the current directory
    if not os.path.exists('data'):
        os.mkdir('data')
    df.to_csv(f'data/{sub_id}_processed.csv', index=False)

In [328]:
process_subject('0001')

C:\Users\osama\AppData\Local\Temp\ipykernel_29892\1221861185.py:18: FutureWarning: pad is deprecated and will be removed in a future version. Use ffill instead.
  df = df.resample('500L', on='seconds').pad()


ValueError: Upsampling from level= or on= selection is not supported, use .set_index(...) to explicitly set index to datetime-like